In [2]:
from random import randint
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 501)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy.stats import norm
from statistics import mean
from math import sqrt
import glob

## Generate Predictions

In [37]:
# Generates n predictions, given parameter values of the population/summary models
def make_n_predictions(n, mu, sigma_dom, sigma_ndom):
    # List of predictions
    pop_acc = []
    sum_acc = []
    pop_p = []
    sum_p = []

    # Generate n predictions
    for i in range(n):
        # Dom and nonDom activations
        D = np.random.normal(0, sigma_dom, 1)
        N = np.random.normal(mu, sigma_ndom, 3)
        
        # Find maximum activation
        allAct = list(D) + list(N)
        M = max(allAct)

        # Discrete uniform J
        J = randint(0,2)
        
        # Population Model
        if D > N[J]:
            pop_acc.append(1)
            pop_p.append(1)
        else:
            pop_acc.append(0)
            pop_p.append(0)

        # Summary Model
        if D == M:
            acc = 1
            p = 1
        elif N[J] == M:
            acc = 0
            p = 0
        elif (D != M) & (N[J] != M):
            acc = np.random.binomial(n=1, p=0.5)
            p = 0.5
        sum_acc.append(acc)
        sum_p.append(p)
    
    # After generating n predictions, append all predictions to a dataframe
    df = pd.DataFrame()
    df['pop_acc'] = pop_acc
    df['sum_acc'] = sum_acc
    df['pop_p'] = pop_p
    df['sum_p'] = sum_p
    return df

# Runs the code to generate predictions
def run_predictions(path, experimentPart):
    # Just some useful strings for naming files
    if experimentPart == 'Yeon-Rahnev':
        end_fname = '_yr_predictions.csv'
        expPart_str = 'yr'
    else:
        end_fname = '_mixed_predictions.csv'
        expPart_str = 'mixed'
    
    # A summary dataframe containing every participant's observed accuracy and predicted accuracies
    all_mean_preds = pd.DataFrame(
        columns=[
            'participant',
            'experimentPart',
            'observed_acc',
            'observed_acc_std',
            'pop_acc',
            'sum_acc',
            'std_pop_acc',
            'std_sum_acc',
            'ttest_pop_ver_obs',
            'ttest_sum_ver_obs',
        ]
    )
    
    # Put all participants' 2-alt data and posterior samples in a list of dataframes
    all_dfs = []
    temp = path + '*_' + expPart_str + '_2-alt-with-dom.csv'
    fnames_2alt = sorted(glob.glob(temp))
    print(fnames_2alt)
    
    temp = '/Users/andrew/Documents/CogSci Thesis/' + expPart_str + '_4-alt_data/model_fitting/*_' + expPart_str + '_posterior_samples.csv'
    fnames_posterior = sorted(glob.glob(temp))
    print(fnames_posterior)

    for f2alt, fpost in zip(fnames_2alt, fnames_posterior):
        df_2alt = pd.read_csv(f2alt)
        df_post = pd.read_csv(fpost)

        # Take every 1000th row of the posterior samples. Any more, and it takes very long
        df_post = df_post.iloc[::1000, :]

        elem = (df_2alt, df_post)
        all_dfs.append(elem)
        
    print('There are', len(all_dfs), 'datasets.')

    # Number of predictions 
    n = 100000
    
    # Generate n predictions for every participant
    for df_2alt, df_post in all_dfs:
        participant = str(df_2alt['participant'].iloc[2])
        
        # Mean parameter values, based on posterior samples
        mu_mean = df_post['mu'].mean()
        sigma_dom_mean = df_post['sigma_dom'].mean()
        sigma_ndom_mean = df_post['sigma_ndom'].mean()
        
        # Generate n predictions and save in csv
        df_pred = make_n_predictions(n, mu_mean, sigma_dom_mean, sigma_ndom_mean)
        fname = path + 'model_predictions/' + participant + end_fname
        df_pred.to_csv(fname)

        # Get accuracies and standard deviations of predictions
        pop_acc = df_pred['pop_acc'].mean()
        sum_acc = df_pred['sum_acc'].mean()
        std_pop_acc = df_pred['pop_acc'].std()
        std_sum_acc = df_pred['sum_acc'].std()

        # Print accuracies and standard deviations of predictions
        print('pop_acc:', pop_acc, ',', 'std:', std_pop_acc)
        print('sum_acc:', sum_acc, ',', 'std:', std_sum_acc)
        display(df_pred)

        # Get actual observed accuracies
        observed = df_2alt['accuracy'].mean()
        observed_std = df_2alt['accuracy'].std()

        # t-tests between observed accuracies and predicted accuracies
        import scipy.stats as stats
        _, pop_ver_obs = stats.ttest_ind(df_2alt['accuracy'], df_pred['pop_acc'])
        _, sum_ver_obs = stats.ttest_ind(df_2alt['accuracy'], df_pred['sum_acc'])

        # Append all relevant information to the summary dataframe
        all_mean_preds.loc[len(all_mean_preds.index)] = [
            participant,
            experimentPart,
            observed,
            observed_std,
            pop_acc,
            sum_acc,
            std_pop_acc,
            std_sum_acc,
            pop_ver_obs,
            sum_ver_obs
        ]

    # Save the summary dataframe
    fname = path + 'model_predictions/all' + end_fname
    all_mean_preds.to_csv(fname)

path = '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/'
experimentPart = 'Yeon-Rahnev'
run_predictions(path, experimentPart)

path = '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/'
experimentPart = 'Mixed'
run_predictions(path, experimentPart)


['/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/10_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/11_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/12_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/13_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/14_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/15_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/16_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/1_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/3_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/4_yr_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/yr_2-alt-with-dom_data/5_yr_2-alt-with-dom.csv', '/Users/andrew/Documents

,pop_acc,sum_acc,pop_p,sum_p
0,1,1,1,1.0
1,1,1,1,1.0
2,1,1,1,1.0
3,1,1,1,1.0
4,1,1,1,1.0
...,...,...,...,...
99995,1,1,1,1.0
99996,1,1,1,1.0
99997,1,1,1,1.0
99998,1,1,1,1.0


pop_acc: 0.80517 , std: 0.39607176095837143
sum_acc: 0.75473 , std: 0.4302493210276465


,pop_acc,sum_acc,pop_p,sum_p
0,1,1,1,0.5
1,0,1,0,0.5
2,1,1,1,1.0
3,1,1,1,1.0
4,1,1,1,1.0
...,...,...,...,...
99995,1,0,1,0.5
99996,1,1,1,1.0
99997,1,1,1,1.0
99998,1,1,1,1.0


pop_acc: 0.75612 , std: 0.4294233221940779
sum_acc: 0.70337 , std: 0.4567742653950554


...

,pop_acc,sum_acc,pop_p,sum_p
0,0,0,0,0.5
1,1,1,1,1.0
2,1,1,1,1.0
3,0,0,0,0.5
4,0,0,0,0.0
...,...,...,...,...
99995,1,1,1,1.0
99996,1,1,1,1.0
99997,1,1,1,1.0
99998,1,1,1,1.0


pop_acc: 0.60308 , std: 0.4892615940056512
sum_acc: 0.57892 , std: 0.49373481884594994


,pop_acc,sum_acc,pop_p,sum_p
0,1,1,1,0.5
1,1,1,1,1.0
2,1,0,1,0.5
3,1,1,1,1.0
4,1,1,1,1.0
...,...,...,...,...
99995,1,1,1,1.0
99996,1,1,1,1.0
99997,1,1,1,1.0
99998,1,1,1,1.0


pop_acc: 0.77775 , std: 0.41576034691491354
sum_acc: 0.7125 , std: 0.4525989377563269


,pop_acc,sum_acc,pop_p,sum_p
0,1,1,1,1.0
1,0,0,0,0.5
2,1,1,1,1.0
3,0,0,0,0.5
4,1,1,1,1.0
...,...,...,...,...
99995,1,1,1,1.0
99996,1,1,1,0.5
99997,1,1,1,1.0
99998,1,1,1,1.0


## Calculate log-likelihood

In [50]:
# Calculates the likelihood of a trial, given a row of posterior samples
def trial_likelihood(y, mu, sigma_dom, sigma_ndom, model):
    # Sigma of X distribution
    new_sigma = sqrt(sigma_dom**2 + sigma_ndom**2)
    
    # Calculate CDFs of X and Z, given row of posterior samples
    X = norm(0-mu, new_sigma).cdf(0)
    Z = norm(0, sigma_ndom*sqrt(2)).cdf(0)
    
    # Population Model
    if model == 'pop':
        p = X*(1/3)
        if y == 0:
            p = 1 - X
      
    # Summary Model
    elif model == 'sum':
        p = X**3 + (1-X)*((Z)**2)*(1/2)
        if y == 0:
            p = 1 - p
     
    # Return trial likelihood
    return p

# Calculates mean likelihood of a trial, across all posterior samples
def mean_trial_likelihood(y, posterior_samples, model):
    # Get columns of posterior samples in order to...
    mu_col = posterior_samples['mu']
    sigma_dom_col = posterior_samples['sigma_dom']
    sigma_ndom_col = posterior_samples['sigma_ndom']
    
    # ... generate a likelihood for every row of the columns
    p = [trial_likelihood(y,mu,sigma_dom,sigma_ndom, model) for mu,sigma_dom,sigma_ndom in zip(mu_col,sigma_dom_col,sigma_ndom_col)]
    
    # Return mean of the likelihoods of a single trial
    return mean(p)
    
# Calculates log-likelihood of a model, given observed data
def log_likelihood(df, posterior_samples, model):
    # Generate mean likelihood for every trial
    df['L_bar'] = [mean_trial_likelihood(y, posterior_samples, model) for y in df['accuracy']]
    
    # Take log of the mean likelihood column
    df['log_L'] = np.log(df['L_bar'])
    display(df)
    
    # Sum the log-likelihood column
    return df['log_L'].sum()

# Runs the likelihood functions
def run_likelihood(path, experimentPart):
    # Useful strings for naming files
    if experimentPart == 'Yeon-Rahnev':
        end_fname = '_yr_predictions.csv'
        expPart_str = 'yr'
    else:
        end_fname = '_mixed_predictions.csv'
        expPart_str = 'mixed'
    
    # Put all participants' 2-alt data and posterior samples in a list of dataframes
    all_dfs = []
    temp = '/Users/andrew/Documents/CogSci Thesis/' + expPart_str + '_2-alt-with-dom_data/' + '*_' + expPart_str + '_2-alt-with-dom.csv'
    fnames_2alt = sorted(glob.glob(temp))
    print(fnames_2alt)
    temp = '/Users/andrew/Documents/CogSci Thesis/' + expPart_str + '_4-alt_data/model_fitting/*_' + expPart_str + '_posterior_samples.csv'
    fnames_posterior = sorted(glob.glob(temp))
    print(fnames_posterior)
    
    for f2alt, fpost in zip(fnames_2alt, fnames_posterior):
        df_2alt = pd.read_csv(f2alt)
        df_post = pd.read_csv(fpost)

        # Take every 1000th row of the posterior samples. Any more, and it takes very long
        df_post = df_post.iloc[::1000, :]

        elem = (df_2alt, df_post)
        all_dfs.append(elem)

    print('There are', len(all_dfs), 'datasets.')

    # A summary dataframe containing every participants' model log-likelihoods, and normalized log-likelihoods
    all_likelihoods = pd.DataFrame(
        columns=[
            'participant', 
            'population_log_likelihood', 
            'summary_log_likelihood',
            'n_2-alt-with-dom_trials',
            'pop_log_likelihood/n',
            'sum_log_likelihood/n',
        ])

    # Generate log-likelihoods for every participant
    for df_2alt, df_post in all_dfs:
        # Log-likelihoods
        pop_log_likelihood = log_likelihood(df_2alt, df_post, model='pop')
        sum_log_likelihood = log_likelihood(df_2alt, df_post, model='sum')

        # Print them
        print(pop_log_likelihood)
        print(sum_log_likelihood)

        # Append all relevant information to summary dataframe
        participant = df_2alt['participant'].iloc[2]
        n_2alt_trials = len(df_2alt.index)
        all_likelihoods.loc[len(all_likelihoods.index)] = [
            participant, 
            pop_log_likelihood, 
            sum_log_likelihood,
            n_2alt_trials,
            pop_log_likelihood/n_2alt_trials,
            sum_log_likelihood/n_2alt_trials
        ]

    # Save summary dataframe
    fname = path + 'log_likelihoods.csv'
    all_likelihoods.to_csv(fname)

path = '/Users/andrew/Documents/CogSci Thesis/yr_4-alt_data/model_fitting/'
run_likelihood(path, 'Yeon-Rahnev')   
    
path = '/Users/andrew/Documents/CogSci Thesis/mixed_4-alt_data/model_fitting/'
run_likelihood(path, 'Mixed')


['/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/10_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/11_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/12_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/13_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/14_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/15_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/16_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/1_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/3_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-with-dom_data/4_mixed_2-alt-with-dom.csv', '/Users/andrew/Documents/CogSci Thesis/mixed_2-alt-w

,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,578,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.117879,-2.138098
1,580,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,0.0,"['green', 'red']",10,0.646363,-0.436394
2,582,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.117879,-2.138098
3,585,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",10,0.117879,-2.138098
4,586,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",10,0.117879,-2.138098
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1137,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",10,0.117879,-2.138098
188,1144,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,0.0,"['white', 'red']",10,0.646363,-0.436394
189,1145,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.117879,-2.138098
190,1149,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['white', 'red']",10,0.646363,-0.436394


,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,578,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.126501,-2.067506
1,580,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,0.0,"['green', 'red']",10,0.873499,-0.135248
2,582,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.126501,-2.067506
3,585,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",10,0.126501,-2.067506
4,586,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",10,0.126501,-2.067506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1137,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",10,0.126501,-2.067506
188,1144,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,0.0,"['white', 'red']",10,0.873499,-0.135248
189,1145,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['green', 'red']",10,0.126501,-2.067506
190,1149,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['white', 'red']",10,0.873499,-0.135248


-349.2533806111822
-327.39985400835945


,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,5,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['red', 'blue']",11,0.075139,-2.588416
1,10,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'green']",11,0.075139,-2.588416
2,15,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",11,0.075139,-2.588416
3,16,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['red', 'blue']",11,0.774583,-0.255430
4,19,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['red', 'blue']",11,0.075139,-2.588416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,560,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,0.0,"['red', 'blue']",11,0.774583,-0.255430
187,561,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['white', 'red']",11,0.075139,-2.588416
188,565,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",11,0.075139,-2.588416
189,566,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",11,0.075139,-2.588416


...

,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,576,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['green', 'red']",8,0.1399,-1.966828
1,584,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'blue']",8,0.1399,-1.966828
2,585,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['green', 'blue']",8,0.1399,-1.966828
3,597,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",8,0.1399,-1.966828
4,599,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",8,0.1399,-1.966828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1142,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",8,0.1399,-1.966828
188,1144,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['green', 'red']",8,0.8601,-0.150706
189,1147,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,1.0,"['white', 'red']",8,0.1399,-1.966828
190,1149,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",8,0.1399,-1.966828


-301.3097963848665
-274.1120756226634


,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,2,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['red', 'blue']",9,0.751034,-0.286304
1,5,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'red']",9,0.082989,-2.489052
2,8,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['green', 'blue']",9,0.082989,-2.489052
3,11,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'green']",9,0.082989,-2.489052
4,14,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",9,0.082989,-2.489052
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,560,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,0.0,"['red', 'blue']",9,0.751034,-0.286304
187,562,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'blue']",9,0.082989,-2.489052
188,566,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['green', 'blue']",9,0.082989,-2.489052
189,571,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",9,0.082989,-2.489052


,Unnamed: 0,part,condition,nWhites,nGreens,nReds,nBlues,answer,accuracy,questionsOptions,participant,p_bar,log_p
0,2,mixed,2-alt-with-dom,11.0,11.0,16.0,11.0,red,0.0,"['red', 'blue']",9,0.889893,-0.116654
1,5,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'red']",9,0.110107,-2.206299
2,8,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['green', 'blue']",9,0.110107,-2.206299
3,11,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'green']",9,0.110107,-2.206299
4,14,mixed,2-alt-with-dom,11.0,16.0,11.0,11.0,green,1.0,"['white', 'green']",9,0.110107,-2.206299
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,560,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,0.0,"['red', 'blue']",9,0.889893,-0.116654
187,562,mixed,2-alt-with-dom,16.0,11.0,11.0,11.0,white,1.0,"['white', 'blue']",9,0.110107,-2.206299
188,566,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['green', 'blue']",9,0.110107,-2.206299
189,571,mixed,2-alt-with-dom,11.0,11.0,11.0,16.0,blue,1.0,"['red', 'blue']",9,0.110107,-2.206299


-415.9347292787835
-364.9827737778074
